## Kaggle – DataTops®
Luismi ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a Luismi a mejorar ese modelo?

## Métrica: 
Error de raíz cuadrada media (RMSE) es la desviación estándar de los valores residuales (errores de predicción). Los valores residuales son una medida de la distancia de los puntos de datos de la línea de regresión; RMSE es una medida de cuál es el nivel de dispersión de estos valores residuales. En otras palabras, le indica el nivel de concentración de los datos en la línea de mejor ajuste.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## Librerías

In [18]:
import numpy as np
import pandas as pd
import toolbox_ML as tb
import bootcampviztools as bt

from PIL import Image
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRFRegressor
from catboost import CatBoostRegressor
import urllib.request

## Datos

In [19]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("./data/train.csv", index_col = 0)
df.index.name = None

In [20]:
df_bck_up = df.copy()

## Exploración de los datos

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB


In [22]:
df.isna().sum()

Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_in_euros      0
dtype: int64

In [23]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [24]:
df.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [25]:
tb.describe_df(df)

,DATA_TYPE,MISSINGS (%),UNIQUE_VALUES,CARDIN (%)
Company,object,0,19,2.08
Product,object,0,480,52.63
TypeName,object,0,6,0.66
Inches,float64,0,17,1.86
ScreenResolution,object,0,36,3.95
Cpu,object,0,107,11.73
Ram,object,0,9,0.99
Memory,object,0,37,4.06
Gpu,object,0,93,10.2
OpSys,object,0,9,0.99


In [26]:
tb.tipifica_variables(df)

,variable,tipo
0,Company,Numérica Discreta
1,Product,Numérica Continua
2,TypeName,Categórica
3,Inches,Numérica Discreta
4,ScreenResolution,Numérica Discreta
5,Cpu,Numérica Discreta
6,Ram,Categórica
7,Memory,Numérica Discreta
8,Gpu,Numérica Discreta
9,OpSys,Categórica


## Procesado de datos

In [27]:
target = "Price_in_euros"

In [28]:
df.drop_duplicates(keep = "first", inplace = True)

In [29]:
df["inches"] = df.Inches.astype(float)
df['ram_gb'] = df['Ram'].str.replace('GB', '').astype(int)
df["weight_kg"] = df['Weight'].str.replace('kg', '').astype(float)
df.drop(columns = ["Inches", "Ram", "Weight"], inplace = True)

In [30]:
feat_sel = tb.get_features_cat_regression(df, "Price_in_euros")
print(feat_sel)

['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'OpSys']


In [31]:
tb.get_features_num_regression(df, "Price_in_euros", 0.6)

[]

In [38]:
columns = ['Company', 'Product', 'TypeName', 'Cpu', 'Memory', 'Gpu', 'inches', 'ram_gb']
columns_dummies = ['Company', 'Product', 'TypeName', 'Cpu', 'Memory', 'Gpu']

In [37]:
df2 = df[columns]

In [39]:
df2 = pd.get_dummies(df2, columns = columns_dummies, dtype = int)

In [40]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 755 to 229
Columns: 744 entries, inches to Gpu_Nvidia Quadro M620M
dtypes: float64(1), int32(743)
memory usage: 2.5 MB


Nuestro target es la columna `Price_in_euros`

-----------------------------------------------------------------------------------------------------------------

## Modelado

### 1. Definir X e y

In [41]:
X = df2
y = df[target]

### 2. Dividir X_train, X_test, y_train, y_test

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [43]:
X_train.head()

,inches,ram_gb,Company_Acer,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,...,Gpu_Nvidia GeForce MX130,Gpu_Nvidia GeForce MX150,Gpu_Nvidia Quadro M1000M,Gpu_Nvidia Quadro M1200,Gpu_Nvidia Quadro M2000M,Gpu_Nvidia Quadro M2200M,Gpu_Nvidia Quadro M3000M,Gpu_Nvidia Quadro M500M,Gpu_Nvidia Quadro M620,Gpu_Nvidia Quadro M620M
112,14.0,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,15.4,16,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1190,15.6,4,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,15.6,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1043,17.3,8,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [44]:
y_train.head()

112     1480.00
13      2439.97
1190     369.00
55       384.00
1043    1080.00
Name: Price_in_euros, dtype: float64

In [45]:
X_train.columns = X_train.columns.str.replace(r'[\[\]<>]', '', regex=True)
X_train.columns = X_train.columns.astype(str)

X_test.columns = X_test.columns.str.replace(r'[\[\]<>]', '', regex=True)
X_test.columns = X_test.columns.astype(str)

### 3. Baseline de modelos


In [46]:
model_names = ["Random Forest", "XGBoost", "Catboost"]
rf_reg = RandomForestRegressor(random_state = 42)
xg_reg = XGBRFRegressor(random_state = 42)
# lgb_reg = LGBMRegressor(random_state = 42, verbose = -100, n_jobs = -1)
cat_reg = CatBoostRegressor(n_estimators = 100, depth = 5, learning_rate = 0.5, verbose = 100)
models = [rf_reg, xg_reg, cat_reg]

resultados = []

for model_name, modelo in zip(model_names, models):
    metrica = np.mean(cross_val_score(modelo, X_train, y_train, cv = 5, scoring  = "neg_root_mean_squared_error"))
    data = {
        "model": model_name,
        "RSME": metrica
    }
    print(data)
    resultados.append(data.copy())
df_resultados = pd.DataFrame(resultados)
df_resultados.sort_values("RSME", ascending = False)

{'model': 'Random Forest', 'RSME': -306.3150718047414}
{'model': 'XGBoost', 'RSME': -341.13395202128834}
0:	learn: 545.3224516	total: 151ms	remaining: 14.9s
99:	learn: 146.7681623	total: 395ms	remaining: 0us
0:	learn: 495.2017750	total: 2.8ms	remaining: 277ms
99:	learn: 137.8323770	total: 251ms	remaining: 0us
0:	learn: 516.7795859	total: 2.95ms	remaining: 292ms
99:	learn: 151.0118174	total: 262ms	remaining: 0us
0:	learn: 536.2130587	total: 3.48ms	remaining: 345ms
99:	learn: 149.6093390	total: 267ms	remaining: 0us
0:	learn: 481.4967007	total: 2.75ms	remaining: 273ms
99:	learn: 143.6477830	total: 247ms	remaining: 0us
{'model': 'Catboost', 'RSME': -296.97530434141675}


,model,RSME
2,Catboost,-296.975304
0,Random Forest,-306.315072
1,XGBoost,-341.133952


In [47]:
cat_reg.fit(X_train, y_train)

0:	learn: 520.7856286	total: 3.57ms	remaining: 353ms
99:	learn: 143.4470276	total: 288ms	remaining: 0us


In [48]:
y_pred = cat_reg.predict(X_test)

In [50]:
# cat_reg = CatBoostRegressor(n_estimators = 100, depth = 5, learning_rate = 0.5, verbose = 100)

cat_reg = CatBoostRegressor(verbose = 100)

param_grid = {
    'iterations': [500, 1000, 2000],  # Cuántos árboles usar
    'learning_rate': [0.01, 0.05, 0.1],  # Tasa de aprendizaje
    'depth': [4, 8, 10],  # Profundidad de los árboles
    'l2_leaf_reg': [1, 4, 10],  # Regularización L2
    'colsample_bylevel': [0.8, 1]  # Porcentaje de features usadas por nivel del árbol
}

grid_model = RandomizedSearchCV(cat_reg, param_distributions = param_grid, n_iter = 20, scoring = "neg_root_mean_squared_error", cv = 4, n_jobs= -1, random_state = 42)

grid_model.fit(X_train, y_train)

print(grid_model.best_score_)
best_model = grid_model.best_estimator_

0:	learn: 643.1898351	total: 15.2ms	remaining: 30.4s
100:	learn: 234.4615205	total: 1.6s	remaining: 30.1s
200:	learn: 175.6823152	total: 3.12s	remaining: 27.9s
300:	learn: 148.1730599	total: 4.58s	remaining: 25.9s
400:	learn: 128.4916494	total: 5.96s	remaining: 23.7s
500:	learn: 114.2763055	total: 7.06s	remaining: 21.1s
600:	learn: 103.2332028	total: 8.09s	remaining: 18.8s
700:	learn: 94.7330445	total: 9.2s	remaining: 17s
800:	learn: 87.2032003	total: 10.3s	remaining: 15.4s
900:	learn: 80.5438072	total: 11.3s	remaining: 13.8s
1000:	learn: 75.1983257	total: 12.3s	remaining: 12.3s
1100:	learn: 70.5056681	total: 13.5s	remaining: 11s
1200:	learn: 65.4831717	total: 14.5s	remaining: 9.67s
1300:	learn: 62.0032818	total: 15.5s	remaining: 8.35s
1400:	learn: 58.7900357	total: 16.6s	remaining: 7.09s
1500:	learn: 56.2790459	total: 17.6s	remaining: 5.84s
1600:	learn: 53.0928206	total: 18.6s	remaining: 4.64s
1700:	learn: 50.4250087	total: 19.7s	remaining: 3.46s
1800:	learn: 48.5554666	total: 20.7s	r

### 4. Sacar métricas, valorar los modelos 

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

In [51]:
y_pred = best_model.predict(X_test)

In [52]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

322.8170917449438


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 1. Carga los datos de `test.csv` para predecir.


In [53]:
X_pred = pd.read_csv("data/test.csv", index_col = 0)
X_pred.index.name = None
X_pred.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg


In [54]:
X_pred.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
820,MSI,GE72MVR 7RG,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
948,Toshiba,Tecra Z40-C-12X,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.47kg
483,Dell,Precision M5520,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg
1017,HP,Probook 440,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.64kg
421,Asus,ZenBook Flip,2 in 1 Convertible,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.27kg


In [55]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   Inches            391 non-null    float64
 4   ScreenResolution  391 non-null    object 
 5   Cpu               391 non-null    object 
 6   Ram               391 non-null    object 
 7   Memory            391 non-null    object 
 8   Gpu               391 non-null    object 
 9   OpSys             391 non-null    object 
 10  Weight            391 non-null    object 
dtypes: float64(1), object(10)
memory usage: 36.7+ KB


 ## 2. Replicar el procesado para ``test.csv``

In [56]:
X_pred["inches"] = X_pred.Inches.astype(float)
X_pred['ram_gb'] = X_pred['Ram'].str.replace('GB', '').astype(int)
X_pred["weight_kg"] = X_pred['Weight'].str.replace('kg', '').astype(float)
X_pred.drop(columns = ["Inches", "Ram", "Weight"], inplace = True)

In [57]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   ScreenResolution  391 non-null    object 
 4   Cpu               391 non-null    object 
 5   Memory            391 non-null    object 
 6   Gpu               391 non-null    object 
 7   OpSys             391 non-null    object 
 8   inches            391 non-null    float64
 9   ram_gb            391 non-null    int32  
 10  weight_kg         391 non-null    float64
dtypes: float64(2), int32(1), object(8)
memory usage: 35.1+ KB


In [58]:
X_pred2 = X_pred[columns]

In [35]:
# # Ordenamos las features categóricas de menor precio de median a mayor precio para poder convertirlas en numéricas:
# categories_company = df.groupby(df["Company"])["Price_in_euros"].median().sort_values().index.to_list()
# # categories_typename = df.groupby(df["TypeName"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_resolution = df.groupby(df["ScreenResolution"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_cpu = df.groupby(df["Cpu"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_memory = df.groupby(df["Memory"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_gpu = df.groupby(df["Gpu"])["Price_in_euros"].median().sort_values().index.to_list()

# total_cat = [categories_company, categories_resolution, categories_cpu, categories_memory, categories_gpu]
# cols = ["Company", "ScreenResolution", "Cpu", "Memory", "Gpu"]

# enc = OrdinalEncoder(categories = total_cat, dtype= int)
# X_pred[cols] = enc.transform(X_pred[cols])

# Con la ram, el sistema operativo y el typename, vamos a hacer un get_dummies:
cols_dummies = X_train.columns
X_pred = pd.get_dummies(X_pred, columns = feat_sel, dtype = int).reindex(columns = cols_dummies, fill_value=0)

In [59]:
cols_dummies = X_train.columns
X_pred2 = pd.get_dummies(X_pred2, columns = columns_dummies, dtype = int).reindex(columns = cols_dummies, fill_value=0)

In [36]:
# X_pred.head()

In [60]:
predictions_submit = best_model.predict(X_pred2)
predictions_submit

array([1711.40177395,  313.89583584,  402.80957049, 1119.82158322,
        943.04563331,  431.15896791,  784.70050447,  925.82326248,
       1254.73367156,  273.89074152, 2030.03985466, 1557.55599849,
        468.21982051, 1652.6665809 , 1137.27645972,  553.89146764,
       2133.21340425, 1282.55546808, 1735.81119215,  576.06061856,
       1735.40879161,  431.75253281,  822.58426836, 1352.69972788,
        439.70094727,  666.44552977,  530.91474156,  740.88811121,
       3140.36624757,  931.90709721, 2251.42754214,  390.18895107,
        942.7378272 , 3486.43578946, 2408.34802616, 1562.65465933,
        629.55456264, 1584.74088836,  884.86512557, 1854.8030987 ,
        700.81263687,  766.99168846,  579.27590387, 1116.16780367,
       1090.19229737, 1069.39591922, 1070.02682125,  659.48716538,
        671.59583425,  424.84262073, 1673.92322933,  710.45761376,
       1035.59256314,  699.89922756, 1808.99272689, 1853.3149925 ,
        590.65166298,  813.92008767,  951.88621749,  597.45784

### 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [61]:
sample = pd.read_csv("data/sample_submission.csv")

In [62]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


In [63]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [64]:
#¿Cómo creamos la submission?
submission = pd.DataFrame({"laptop_ID": X_pred.index,
                        "Price_in_euros": predictions_submit})

In [65]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1711.401774
1,1281,313.895836
2,1168,402.809570
3,1231,1119.821583
4,1020,943.045633


In [66]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [67]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [68]:
chequeador(submission)

You're ready to submit!
